In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import re
import string
import os
from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop = set(stopwords.words('english'))
import gensim
from tqdm.notebook import tqdm
import nltk
nltk.download('wordnet')
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
import logging
from copy import deepcopy
def seed_everything(seed=27):
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed_everything()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def remove_unnecessary(text):
    #remove_URL
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub('', text)

    #remove_html
    html = re.compile(r'<.*?>')
    text = html.sub('', text)

    #remove_emoji
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    #remove_punct
    table = str.maketrans('', '', string.punctuation)
    text = text.translate(table)

    #lemmatize_sentence
    sentence_words = text.split(' ')
    new_sentence_words = list()
    
    for sentence_word in sentence_words:
        sentence_word = sentence_word.replace('#', '')
        new_sentence_word = WordNetLemmatizer().lemmatize(sentence_word.lower(), wordnet.VERB)
        new_sentence_words.append(new_sentence_word)
        
    new_sentence = ' '.join(new_sentence_words)
    new_sentence = new_sentence.strip()

    return new_sentence


def prepare_csv(df_train,seed=27, val_ratio=0.2):
    idx = np.arange(df_train.shape[0])    
    np.random.shuffle(idx)
    #print('idx',idx[0])
    val_size = int(len(idx) * val_ratio)
    if not os.path.exists('cache'): # cache is tem memory file 
        os.makedirs('cache')
    
    df_train.iloc[idx[val_size:], :][['id', 'sentiment', 'review']].to_csv(
        'cache/dataset_train.csv', index=False
    )
    
    df_train.iloc[idx[:val_size], :][['id', 'sentiment', 'review']].to_csv(
        'cache/dataset_val.csv', index=False
    )
    
    #df_test[['id', 'text']].to_csv('cache/dataset_test.csv',
                   #index=False)
    
def get_iterator(dataset, batch_size, train=True,
                 shuffle=True, repeat=False):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
    dataset_iter = data.Iterator(
        dataset, batch_size=batch_size, device=device,
        train=train, shuffle=shuffle, repeat=repeat,
        sort=False)  
    return dataset_iter

def get_dataset(fix_length=100, lower=False, vectors=None,train_dir = 'sentiment.tsv'):
    #sample_submission = pd.read_csv(sample_dir)
    #test = pd.read_csv(test_dir)
    train = pd.read_csv(train_dir,sep='\t',header=0)
    train['review'] = train['review'].apply(lambda x: remove_unnecessary(x))
    #test['text'] = test['text'].apply(lambda x: remove_unnecessary(x))
    if vectors is not None:
        lower=True

    prepare_csv(train)
    
    REVIEW = data.Field(sequential=True, 
#                     tokenize='spacy', 
                      lower=True, 
                      include_lengths=True, 
                      batch_first=True, 
                      fix_length=fix_length)
    SENTIMENT = data.Field(use_vocab=True,
                       sequential=False,
                       dtype=torch.float16)
    ID = data.Field(use_vocab=False,
                    sequential=False,
                    dtype=torch.float16)   
    train_temp, val_temp = data.TabularDataset.splits(
        path='cache/', format='csv', skip_header=True,
        train='dataset_train.csv', validation='dataset_val.csv',
        fields=[
            ('id', ID),
            ('sentiment', SENTIMENT),
            ('review', REVIEW)])  
  
    #test_temp = data.TabularDataset(
        #path='cache/dataset_test.csv', format='csv',
        #skip_header=True,
        #fields=[
           # ('id', ID),
           # ('text', TEXT)])  
  
    REVIEW.build_vocab(
        train_temp, val_temp, 
        max_size=20000,
        min_freq=10,
        vectors=GloVe(name='6B', dim=300)  # We use it for getting vocabulary of words
    )
    SENTIMENT.build_vocab(
        train_temp,val_temp
    )
    ID.build_vocab(
        train_temp, val_temp
    )
    
    word_embeddings = REVIEW.vocab.vectors
    vocab_size = len(REVIEW.vocab)
    
    train_iter = get_iterator(train_temp, batch_size=32, 
                              train=True, shuffle=True,
                              repeat=False)
    val_iter = get_iterator(val_temp, batch_size=32, 
                            train=True, shuffle=True,
                            repeat=False)
    #test_iter = get_iterator(test_temp, batch_size=32, 
                            # train=False, shuffle=False,
                            # repeat=False)  
 
    return vocab_size, word_embeddings, train_iter, val_iter


In [0]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, weights):
        super(LSTMClassifier, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = torch.nn.Embedding(vocab_size,
                                                  embedding_dim)
        self.word_embeddings.weight = torch.nn.Parameter(weights,
                                                         requires_grad=False)
        
        self.dropout_1 = torch.nn.Dropout(0.3)
        self.lstm = torch.nn.LSTM(embedding_dim,
                                  hidden_dim,
                                  n_layers,
                                  dropout=0.3,
                                  batch_first=True)
        
        self.dropout_2 = torch.nn.Dropout(0.3)
        self.label_layer = torch.nn.Linear(hidden_dim, output_size)
        
        self.act = torch.nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        
        x = self.word_embeddings(x)
        
        x = self.dropout_1(x)
        
        lstm_out, hidden = self.lstm(x, hidden)
                
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout_2(lstm_out)
        out = self.label_layer(out)    
        
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1, :]

        out = self.act(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        
        return hidden

def train_model(model, train_iter, val_iter, optim, loss, num_epochs, batch_size=32):
    h = model.init_hidden(batch_size)
    
    clip = 5
    val_loss_min = np.Inf
    
    total_train_epoch_loss = list()
    total_train_epoch_acc = list()
        
    total_val_epoch_loss = list()
    total_val_epoch_acc = list()
        
    
    device = torch.device('cuda:0' if torch.cuda.is_available()
                           else 'cpu')
    best_epoch = 0
    best_acc = 0 
    for epoch in range(num_epochs):

        model.train()
        
        train_epoch_loss = list()
        train_epoch_acc = list()
        
        val_epoch_loss = list()
        val_epoch_acc = list()
        
        for idx, batch in enumerate(tqdm(train_iter)):
            h = tuple([e.data for e in h])

            review = batch.review[0]
            sentiment = batch.sentiment
            sentiment = sentiment - 1
            sentiment = sentiment.type(torch.LongTensor)

            review = review.to(device)
            sentiment = sentiment.to(device)

            optim.zero_grad()
            
            if review.size()[0] is not batch_size:
                continue
            
            prediction, h = model(review, h)
                
            loss_train = loss(prediction.squeeze(), sentiment)
            loss_train.backward()

            num_corrects = (torch.max(prediction, 1)[1].
                                view(sentiment.size()).data == sentiment.data).float().sum()

            acc = 100.0 * num_corrects / len(batch)

            train_epoch_loss.append(loss_train.item())
            train_epoch_acc.append(acc.item())
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            
            optim.step()
    
        print(f'Train Epoch: {epoch}, Training Loss: {np.mean(train_epoch_loss):.4f}, Training Accuracy: {np.mean(train_epoch_acc): .2f}%')

        model.eval()

        with torch.no_grad():
            for idx, batch in enumerate(tqdm(val_iter)):
                val_h = tuple([e.data for e in h])

                review = batch.review[0]
                sentiment = batch.sentiment
                sentiment = sentiment - 1
                sentiment = sentiment.type(torch.LongTensor)
                
                review = review.to(device)
                sentiment = sentiment.to(device)
                
                if review.size()[0] is not batch_size:
                    continue

                prediction, h = model(review, h)
                loss_val = loss(prediction.squeeze(), sentiment)

                num_corrects = (torch.max(prediction, 1)[1].
                                view(sentiment.size()).data == sentiment.data).float().sum()

                acc = 100.0 * num_corrects / len(batch)

                val_epoch_loss.append(loss_val.item())
                val_epoch_acc.append(acc.item())
                
            print(f'Vadlidation Epoch: {epoch}, Training Loss: {np.mean(val_epoch_loss):.4f}, Training Accuracy: {np.mean(val_epoch_acc): .2f}%')
                
            if np.mean(val_epoch_acc) >= best_acc:
                best_acc = np.mean(val_epoch_acc)
                best_epoch = epoch
                torch.save(model.state_dict(), 'state_dict.pth')
                print('Validation loss decreased ({:.6f} --> {:.6f})'.
                      format(val_loss_min, np.mean(val_epoch_loss)))
                val_loss_min = np.mean(val_epoch_loss)
            print('Epoch:%d, acc:%.4f best_epoch:%d best_acc:%.4f'%(epoch,np.mean(val_epoch_acc),best_epoch, best_acc))
    
            

        total_train_epoch_loss.append(np.mean(train_epoch_loss))
        total_train_epoch_acc.append(np.mean(train_epoch_acc))
    
        total_val_epoch_loss.append(np.mean(val_epoch_loss))
        total_val_epoch_acc.append(np.mean(val_epoch_acc))
    
    return (total_train_epoch_loss, total_train_epoch_acc,
            total_val_epoch_loss, total_val_epoch_acc)



In [23]:
lr = 1e-4
batch_size = 32
output_size = 2
hidden_size = 128
embedding_length = 300
#seed_everything(seed=27)

vocab_size, word_embeddings, train_iter, val_iter = get_dataset(fix_length=25,train_dir='sentiment.tsv')

model = LSTMClassifier(vocab_size=vocab_size, 
                       output_size=output_size, 
                       embedding_dim=embedding_length,
                       hidden_dim=hidden_size,
                       n_layers=2,
                       weights=word_embeddings
)

device = torch.device('cuda:0' if torch.cuda.is_available()
                      else 'cpu')
    
model.to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()
    
train_loss, train_acc, val_loss, val_acc = train_model(model=model,train_iter=train_iter,val_iter=val_iter,optim=optim,loss=loss,num_epochs=30,batch_size=batch_size)

results_target = list()
model.load_state_dict(torch.load("state_dict.pth"))





Train Epoch: 0, Training Loss: 0.6731, Training Accuracy:  57.06%



Vadlidation Epoch: 0, Training Loss: 0.6412, Training Accuracy:  64.54%
Validation loss decreased (inf --> 0.641238)
Epoch:0, acc:64.5433 best_epoch:0 best_acc:64.5433



Train Epoch: 1, Training Loss: 0.6261, Training Accuracy:  65.53%



Vadlidation Epoch: 1, Training Loss: 0.6245, Training Accuracy:  65.95%
Validation loss decreased (0.641238 --> 0.624465)
Epoch:1, acc:65.9455 best_epoch:1 best_acc:65.9455



Train Epoch: 2, Training Loss: 0.6152, Training Accuracy:  66.91%


...


Train Epoch: 28, Training Loss: 0.5349, Training Accuracy:  76.53%



Vadlidation Epoch: 28, Training Loss: 0.5830, Training Accuracy:  70.79%
Epoch:28, acc:70.7933 best_epoch:25 best_acc:71.7348



Train Epoch: 29, Training Loss: 0.5284, Training Accuracy:  77.19%



Vadlidation Epoch: 29, Training Loss: 0.5789, Training Accuracy:  71.39%
Epoch:29, acc:71.3942 best_epoch:25 best_acc:71.7348


<All keys matched successfully>